In [20]:
import pandas as pd
import numpy as np
import math

import io

def process_pages(page):
      title_or_name="name"
      not_claim=page+"_"
      if page=="claim":
        title_or_name="title"
        not_claim=""
      all_sections = pd.read_csv("q - "+page+"_review.csv")
      all_sections["sec_number"] = all_sections["sec_number"].astype(str)

      unique_sections=all_sections.sec_number.unique()

      tmp=all_sections[['sec_number','q_title']].groupby(['sec_number']).agg(['count'])
      sec_size={}
      variables={}
      for s in unique_sections:
          sec_size[s]=tmp.query("sec_number=='"+s+"'").iloc[0]['q_title']['count']
          
      max=len(unique_sections)
      last_sec=""
      validation=""
      field=""
      file_contents="<p id='notice'><%= notice %></p>"
      buttons=""
      questions=""
      i=0
      for index, row in all_sections.iterrows():
          row['variable_name']=str(row['variable_name'])
          if last_sec!=row['sec_number']:
              last_sec=row['sec_number']
              req=""
          i=i+1
          required=""
        
          if row['is_required']:
              req="<span class='required'>* <%= t('required') %></span>"
#          print ("\n-select-\n"+str(row['variable_name'])+" \n-"+str(row['text_field_type'])+"-\n "+str(row['options']))
          if row['text_field_type']=="text_field":
              field="<%= f.text_field :"+row['variable_name']+", class: 'field-long', value: @"+page+"_review."+row['variable_name']+", disabled: ENV['BLOCKCHAIN_ENABLED']=='true' || @"+page+"_review.user_id!=current_user.id %>"
          elif row['text_field_type']=="textarea":
              field="<%= f.text_area(:"+row['variable_name']+", size: '50x10', class: 'field-long field-textarea', value:@"+page+"_review."+row['variable_name']+", disabled: ENV['BLOCKCHAIN_ENABLED']=='true' || @"+page+"_review.user_id!=current_user.id) %>"
          elif row['text_field_type']=="select":
              print ("select!")
              options=row['options'].split(',')
              options_val=str(row['options_val']).split(' ')
              maxl=len(options)
              sel_options=""
              for ix in range(maxl):
                  sel_options=sel_options+"[t('"+options[ix]+"').sub('<b>','').sub('</b>',''), "+str(options_val[ix])+"],"
              sel_options=sel_options.rstrip(',')
              field="<%= f.select :"+row['variable_name']+", options_for_select([['Select',''],"+sel_options+"], :selected => @"+page+"_review."+row['variable_name']+"), {}, disabled: ENV['BLOCKCHAIN_ENABLED']=='true' || @"+page+"_review.user_id!=current_user.id %>"
          elif row['text_field_type']=="radio":
              options=row['options'].split(',')
              options_val=str(row['options_val']).split(' ')
              print ("\n-select-\n"+str(row['variable_name']))
              maxl=len(options)
              sel_options=""
              for ix in range(maxl):
                  sel_options=sel_options+"[t('"+options[ix]+"').sub('<b>','').sub('</b>',''), "+str(options_val[ix])+"],"
              sel_options=sel_options.rstrip(',')
              field="<%= f.select :"+row['variable_name']+", options_for_select([['Select',''],"+sel_options+"], :selected => @"+page+"_review."+row['variable_name']+"), {}, disabled: ENV['BLOCKCHAIN_ENABLED']=='true' || @"+page+"_review.user_id!=current_user.id %>"
          else:
              field="<%= f.hidden_field :"+page+"_review, :value => @"+page+"_review %>"

          if (len(row['variable_name'])>3 and not row['variable_name'].endswith("review_started")):
              if (not row['is_required']):
                no_blank="!@"+page+"_review."+row['variable_name']+".blank? AND "
              else:
                no_blank=""
              blockchain_condition1=""
              blockchain_condition2=""
              if (str(row['variable_name'])=="add_to_blockchain"):
                blockchain_condition1="\n             <% if ENV['BLOCKCHAIN_ENABLED']=='true' %>\n"
                blockchain_condition2="\n             <% end %>\n"
              questions=questions+'''
                  {b1}<% if (@{p}_review.user_id==current_user.id or @{p}_review.{nc}review_sharing_mode==1 or current_user.role=="admin") and !@{p}_review.{v}.blank? %>
                      <li><b>*</b> {question}{required}
                      <br><b>{field}</b>
                      <br><hr>
                      </li>
                  <% end %>{b2}
              '''.format(nb=no_blank,nc=not_claim,p=page,question="<%== t('"+row['q_title']+"') %>",required=required,field=field,v=row['variable_name'],count=i,b1=blockchain_condition1,b2=blockchain_condition2)
          if (str(row['variable_name'])=="add_to_blockchain"):
            print ("\n----\nChecking:\n"+questions+"\n-----\n")
          buttons="<% if ENV['BLOCKCHAIN_ENABLED']!='true' && (@"+page+"_review.user_id == current_user.id or current_user.role=='admin') %>\n    <%= f.submit t('update'), :class=>'btn btn btn-primary' %>\n<% end %>\n"

          file_content='''
              <div class="form-claim">
              <h6><%= t('showing_reviews_for') %>{p}:<br>
                   <%= link_to @{p}.{title_or_name}, {p}_path(@{p}.id) %>
                   <hr><p><%= t('reviewed_by') %>: <%= User.find(@{p}_review.user_id).name %>\n<br><%= t('created_at') %>: <%= @{p}_review.created_at %>
                   <br><%= t('updated_at') %>: <%= @{p}_review.updated_at %>

                    <% if ENV['BLOCKCHAIN_ENABLED']!='true' && (@{p}_review.user_id == current_user.id || current_user.role=="admin")  %>
                      <br><br><%= link_to t('edit_review'), {p}_{p}_reviews_path(@{p}.id), method: :create %> -
                      <%= link_to t('del_review'), {p}_{p}_review_path(@{p}.id,@{p}_review), method: :delete, data: {{ confirm: t('warning_del') }} %>
                    <% end %>

                   </p>
              </h6> 

              <%= form_for @{p}_review, method: :put, url: {p}_{p}_review_path(@{p},@{p}_review) do |f| %>
              <% if f.object.errors.any? %>
              <div class="error_messages">
                  <% f.object.errors.full_messages.each do |error| %>
                    <p><%= error %></p>
                  <% end %>
              </div>
              <% end %>
              <fieldset>
                  <ul>
                      {qs}
                    <li>
                      {b}
                    </li>
                  </ul>

                  <% if (ENV['BLOCKCHAIN_ENABLED']=='true' && @claim_review.blockchain_tx.present?) %>
                    <div id="assessment"></div>
                    <div id="assessment_div">
                      <hr>
                      <h4><%= t('assess_factcheck') %></h4><br>
                      <% if @claim_review.blockchain_assessment.nil?%>
                          <% if current_user.role=="admin" %>
                              <%= t('assessment') %>:
                              <select name="blockchain_assessment" id="blockchain_assessment">
                                <option value="1">Accepted</option>
                                <option value="0">Rejected</option>
                              </select><br>
                              <br><%= t('assessment_rationale') %>:<br><textarea id="assessment_rationale" name="assessment_rationale" rows="4" cols="50"></textarea>
                              <br><button name="asssmentButton" type="button" value="<% t('submit') %>" onclick=javascript:assess("<%= config.relative_url_root.to_s %>","<%= config.relative_url_root.to_s+"/claims/"+@claim.id.to_s+"/claim_reviews/"+@claim_review.id.to_s %>"); return false;><%= t('submit') %></button>
                          <% end %>
                      <% else %>
                        <%= t('assessment') %>:
                            <% if @claim_review.blockchain_assessment==1 %>
                              <%= t('accepted') %>
                            <% else %>
                              <%= t('rejected') %>
                            <% end %>
                            <% if @claim_review.blockchain_assessment_rationale.present? %>
                              <br><br><b><%= t('assessment_rationale') %>:</b> <%= @claim_review.blockchain_assessment_rationale %>
                            <% end %>
                      <% end %>
                    </div>
                  <% end %>

              </fieldset>
              <% end %>
          '''.format(s="['"+"', '".join(map(str,unique_sections))+"']",sn=row['stage_number'],st="<%== t('"+row['stage_title']+"') %>",qs=questions,b=buttons,p=page,title_or_name=title_or_name)

          file_content=file_content+"</div>"
          print ("\n\n\n../app/views/"+page+"_reviews/show.html.erb\n\n\n")
          with io.open("../app/views/"+page+"_reviews/show.html.erb", 'w', encoding='utf8') as f:
              f.write(file_content)

      print ("-------------------"+page+"_reviews/show--------------------------\n")
    


In [21]:
process_pages("medium")


-select-
medium_is_blacklisted



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_failed_factcheck_before



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_has_poor_security



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_whois_info_discrepency



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_hosting_discrepency



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_is_biased



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_is_poorly_ranked



../app/views/medium_reviews/show.html.erb






../app/views/medium_reviews/show.html.erb




-select-
medium_is_not_liable



../app/views/medium_reviews/show.html.erb








In [22]:
process_pages("src")


-select-
src_lacks_proper_credentials



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_failed_factcheck_before



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_has_poor_writing_history



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_supported_by_trolls



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_difficult_to_locate



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_other_problems



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_review_verdict



../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb






../app/views/src_reviews/show.html.erb




-select-
src_review_sharing_mode



.

In [23]:
process_pages("claim")


-select-
img_review_started



../app/views/claim_reviews/show.html.erb




-select-
img_old



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
img_forensic_discrepency



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
img_metadata_discrepency



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
img_logical_discrepency



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
vid_review_started



../app/views/claim_reviews/show.html.erb




-select-
vid_old



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
vid_forensic_discrepency



../app/views/claim_reviews/show.html.erb






../app/views/claim_reviews/show.html.erb




-select-
vid_metadata_discrepency



../app/views/claim_reviews/show.html.erb






../app/views/claim_revie